In [1]:
# 라이브러리 import
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities #다음화면으로 빨리 넘어갈때
from selenium.common.exceptions import UnexpectedAlertPresentException #selenium 이용시 Alert을 제어
import urllib.request
import random
from selenium.webdriver.chrome.options import Options

In [2]:
# 크롤링 시작과 끝 날짜 (달 별로 크롤링)

first_days = pd.date_range('2017/01/01', '2022/11/14',freq = 'MS') # 각 달의 첫 날
last_days = pd.date_range('2017/01/01', '2022/11/14', freq = 'M') # 각 달의 마지막 날

keyword_list = ['1인 밀키트', '독립일기', '자취방구하기']

In [3]:
# 크롬 드라이버 제어 
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') # bypass OS security model
chrome_options.add_argument('--disable-dex-shm-usage')
# options.add_argument('--headless') # 브라우저 백그라운드 모드

#현재 컴퓨터 크롬 드라이버 위치
chrome_path = r"C:\Users\user\chromedriver.exe"

In [4]:
# 블로그 주소 수집
blog_url_list = []

# 네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
    driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
    driver.implicitly_wait(3)
    
    driver.get(url)
    time.sleep(3)
    
    url_want = 990
    
    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() # 검색 옵션 클릭
    
    # 날짜 지정
    for k in range(len(first_days)):
        try:
            # 시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()
            
            # 스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            
            scroll = (url_want/30) - 1
            
            #스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
                time.sleep(2)
                
                new_height = driver.execute_script('return document.body.scrollHeight')
                
                if new_height == last_height:
                    break
                
                last_height = new_height
                
            soup = bs(driver.page_source, 'lxml')
            blog_url = soup.find_all('a', class_='api_txt_lines total_tit')
            
            # url 주소 수집
            for i in blog_url:
                blog_url_list.append(i["href"]) # 주소 수집
            print('{0} 키워드 {1}년 {2}월'.format(keyword, first_days.year[k], first_days.month[k]), len(blog_url),'개 블로그 url 주소 수집')

            driver.execute_script('window.scrollTo(0,0)')
            
        except:
            pass

1인 밀키트 키워드 2017년 1월 2 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 2월 1 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 3월 0 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 4월 2 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 5월 3 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 6월 2 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 7월 5 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 8월 2 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 9월 4 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 10월 6 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 11월 4 개 블로그 url 주소 수집
1인 밀키트 키워드 2017년 12월 7 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 1월 15 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 2월 16 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 3월 14 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 4월 17 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 5월 25 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 6월 10 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 7월 18 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 8월 17 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 9월 20 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 10월 29 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 11월 30 개 블로그 url 주소 수집
1인 밀키트 키워드 2018년 12월 26 개 블로그 url 주소 수집
1인 밀키트 키워드 2019년 1월 31 개 블로그 url 주소 수집
1인 밀키트 키워드 2019년 2월 39 개 블로그 ur

자취방구하기 키워드 2022년 10월 302 개 블로그 url 주소 수집


In [5]:
len(blog_url_list)

94906

In [6]:
# 수집 후 url list 중복 확인

real_list = set(blog_url_list)
blog_url_list = list(real_list)
len(blog_url_list)

93982

In [7]:
# 파일 저장(블로그 주소)
k = pd.DataFrame(blog_url_list)
k.to_csv('url_list_1인가구.csv', encoding="UTF-8-sig")

In [8]:
# 블로그 크롤링
url = pd.read_csv('url_list_1인가구.csv')
blog_url_list = list(url['0'])

In [9]:
no_naver_blog1=[]

start = time.time()


time_list = [] #시간
review_list = [] #리뷰
comment_list = [] #댓글
like_list = [] #좋아요수
url_list = [] #url주소
count = 0

driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
for url in blog_url_list: #기본적으로 고치지 않으시지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고칩니다.
    if 'naver' in url: #네이버 블로그만 수집(다른 사이트가 섞이면 크롤링 에러가 남)
    
        count += 1
        driver.get(url) #url 하나씩 글을 수집할꺼야
        time.sleep(1)

        try:
            # 전체 본문 가지고 와서
            driver.switch_to.frame('mainFrame') #네이버는 mainFrame을 바꿔줘야 수집이 됌
            time.sleep(1)
        
            soup = bs(driver.page_source, 'lxml') #뷰티풀숩을 이용해서 구문 분석(lxml 파싱 이용. 모든 테그에서 데이터를 추출위함)
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]') #r은 raw string. \s+는 공백문자
            postview_reg = reg.sub(' ',postview) #공백을 띄어쓰기로 대치해서 본문 글 수집
            try:
                # 좋아요수
                like = driver.find_element("xpath", "//span[@class='u_likeit_list_btn _button btn_sympathy pcol2 off']/em[@class='u_cnt _count']")
                like = like.text
            except:
                like = [] #좋아요 없으면 빈 리스트
            
            try:
                timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") #수정 안 한 경우
        
            except:
                timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") #수정 한 경우
            timeline = timeline.text

            try:
                driver.find_element(By.CSS_SELECTOR,'span.btn_arr').click() #댓글 내리는 버튼(태그명.클래스선택자)
                #driver.find_element_by_css_selector('span.btn_arr').click()
                time.sleep(1.7)

                comment_blog = []
                comment = driver.find_element(By.CSS_SELECTOR,'span.u_cbox_contents') #댓글
                #comment = driver.find_elements_by_css_selector('span.u_cbox_contents')
                for i in comment: #댓글 수집
                    com = str(i.text) #
                    com = reg.sub(' ',com) 
                    comment_blog.append(([com]))
            except: 
                comment_blog = []
             
            time_list.append(timeline)
            review_list.append(postview_reg)
            comment_list.append(comment_blog)
            url_list.append(url)
            like_list.append(like)
            time.sleep(random.uniform(1,1.6))
        except UnexpectedAlertPresentException:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%10 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))

C:\Users\user\AppData\Local\Temp\ipykernel_20324\3172629000.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.


10개 블로그 크롤링 완료
20개 블로그 크롤링 완료
30개 블로그 크롤링 완료
40개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://www.kinocine.com/3292
50개 블로그 크롤링 완료
네이버 블로그 아닌 url:  http://class1.co.kr/9515821873
60개 블로그 크롤링 완료
70개 블로그 크롤링 완료
80개 블로그 크롤링 완료
90개 블로그 크롤링 완료
100개 블로그 크롤링 완료
110개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://aibot.tistory.com/4529
120개 블로그 크롤링 완료
130개 블로그 크롤링 완료
140개 블로그 크롤링 완료
150개 블로그 크롤링 완료
160개 블로그 크롤링 완료
네이버 블로그 아닌 url:  http://blog.jinbo.net/8434pjr/492
170개 블로그 크롤링 완료
180개 블로그 크롤링 완료
190개 블로그 크롤링 완료
200개 블로그 크롤링 완료
210개 블로그 크롤링 완료
220개 블로그 크롤링 완료
230개 블로그 크롤링 완료
240개 블로그 크롤링 완료
250개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://aridasom.tistory.com/1396
260개 블로그 크롤링 완료
270개 블로그 크롤링 완료
280개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://1review.tistory.com/47
290개 블로그 크롤링 완료
300개 블로그 크롤링 완료
310개 블로그 크롤링 완료
320개 블로그 크롤링 완료
330개 블로그 크롤링 완료
340개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://sevenstones.tistory.com/187
350개 블로그 크롤링 완료
360개 블로그 크롤링 완료
370개 블로그 크롤링 완료
380개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://thisisyoung.tistory.com/125
39

2430개 블로그 크롤링 완료
2440개 블로그 크롤링 완료
2450개 블로그 크롤링 완료
2460개 블로그 크롤링 완료
2470개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://eunbit1678.tistory.com/105
2480개 블로그 크롤링 완료
2490개 블로그 크롤링 완료
2500개 블로그 크롤링 완료
2510개 블로그 크롤링 완료
2520개 블로그 크롤링 완료
네이버 블로그 아닌 url:  http://class1.co.kr/4562816279
네이버 블로그 아닌 url:  https://bapbancahn.tistory.com/371
2530개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://kangdbang.tistory.com/1031
2540개 블로그 크롤링 완료
2550개 블로그 크롤링 완료
네이버 블로그 아닌 url:  http://slownews.kr/62215
2560개 블로그 크롤링 완료
2570개 블로그 크롤링 완료
2580개 블로그 크롤링 완료
네이버 블로그 아닌 url:  http://class1.co.kr/7681076971
2590개 블로그 크롤링 완료
2600개 블로그 크롤링 완료
2610개 블로그 크롤링 완료
2620개 블로그 크롤링 완료
2630개 블로그 크롤링 완료
2640개 블로그 크롤링 완료
2650개 블로그 크롤링 완료
2660개 블로그 크롤링 완료
2670개 블로그 크롤링 완료
2680개 블로그 크롤링 완료
2690개 블로그 크롤링 완료
2700개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://www.chemidream.com/1811?category=390443
2710개 블로그 크롤링 완료
2720개 블로그 크롤링 완료
2730개 블로그 크롤링 완료
2740개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://gomubandlife.tistory.com/114
2740개 블로그 크롤링 완료
2750개 블로그 크롤링 완료
네이버 블로그 아

4350개 블로그 크롤링 완료
4360개 블로그 크롤링 완료
4370개 블로그 크롤링 완료
네이버 블로그 아닌 url:  http://coocar.co.kr/8631052044
네이버 블로그 아닌 url:  https://wolfpack.tistory.com/1180672202
4380개 블로그 크롤링 완료
4390개 블로그 크롤링 완료
4400개 블로그 크롤링 완료
4410개 블로그 크롤링 완료
4420개 블로그 크롤링 완료
4430개 블로그 크롤링 완료
4440개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://kangdbang.tistory.com/1078
네이버 블로그 아닌 url:  https://now4here.tistory.com/183
4450개 블로그 크롤링 완료
4460개 블로그 크롤링 완료
4470개 블로그 크롤링 완료
4480개 블로그 크롤링 완료
4490개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://blackre.tistory.com/42
4490개 블로그 크롤링 완료
4500개 블로그 크롤링 완료
4510개 블로그 크롤링 완료
4520개 블로그 크롤링 완료
4530개 블로그 크롤링 완료
4540개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://bottegayun.tistory.com/546
4550개 블로그 크롤링 완료
4560개 블로그 크롤링 완료
4570개 블로그 크롤링 완료
4580개 블로그 크롤링 완료
4590개 블로그 크롤링 완료
4600개 블로그 크롤링 완료
4610개 블로그 크롤링 완료
4620개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://jinyong7269.tistory.com/51
4620개 블로그 크롤링 완료
4630개 블로그 크롤링 완료
4640개 블로그 크롤링 완료
4650개 블로그 크롤링 완료
4660개 블로그 크롤링 완료
4670개 블로그 크롤링 완료
4680개 블로그 크롤링 완료
4690개 블로그 크롤링 완료
네이버 블로그 아닌 u

6370개 블로그 크롤링 완료
6380개 블로그 크롤링 완료
6390개 블로그 크롤링 완료
6400개 블로그 크롤링 완료
6410개 블로그 크롤링 완료
6420개 블로그 크롤링 완료
6430개 블로그 크롤링 완료
6440개 블로그 크롤링 완료
6450개 블로그 크롤링 완료
6460개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://kangsunseng.tistory.com/149
6470개 블로그 크롤링 완료
6480개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://tigerbooks.tistory.com/105
6480개 블로그 크롤링 완료
6490개 블로그 크롤링 완료
6500개 블로그 크롤링 완료
6510개 블로그 크롤링 완료
6520개 블로그 크롤링 완료
6530개 블로그 크롤링 완료
6540개 블로그 크롤링 완료
6550개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://free2world.tistory.com/2424
6560개 블로그 크롤링 완료
6570개 블로그 크롤링 완료
6580개 블로그 크롤링 완료
6590개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://www.urimal.org/1452
6600개 블로그 크롤링 완료
6610개 블로그 크롤링 완료
6620개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://shwank77.tistory.com/1309
6630개 블로그 크롤링 완료
6640개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://qq9447.tistory.com/339
네이버 블로그 아닌 url:  https://parksunny.tistory.com/110
6650개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://neodol.tistory.com/2953
네이버 블로그 아닌 url:  https://www.read1825.com/entry/6%EC%9B%94%EC%9D%98-%EC%B6%94%EC%B2%9C%E

...

12290개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://polystar.tistory.com/183
12300개 블로그 크롤링 완료
12310개 블로그 크롤링 완료
12320개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://parksungwoong.tistory.com/35643
네이버 블로그 아닌 url:  http://stylework.co.kr/19720544
12330개 블로그 크롤링 완료
12340개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://osunji.tistory.com/2645
12350개 블로그 크롤링 완료
12360개 블로그 크롤링 완료
12370개 블로그 크롤링 완료
12380개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://crayonjh.tistory.com/41
12390개 블로그 크롤링 완료
12400개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://sweet-myo.tistory.com/915
12410개 블로그 크롤링 완료
12420개 블로그 크롤링 완료
12430개 블로그 크롤링 완료
12440개 블로그 크롤링 완료
12450개 블로그 크롤링 완료
12460개 블로그 크롤링 완료
12470개 블로그 크롤링 완료
12480개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://re-rock.tistory.com/346
12490개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://kketip.tistory.com/117
12500개 블로그 크롤링 완료
12510개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://giannistips.tistory.com/entry/%EB%B0%80%ED%82%A4%ED%8A%B8-%EB%9C%BB-%EB%B0%80%ED%82%A4%ED%8A%B8-%EC%97%B4%ED%92%8D-%EB%8B%A4%EC%9D%B4%EC%96%B4%ED%8A%B8%EC%97%9

14310개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://dailypleasure.tistory.com/304
14310개 블로그 크롤링 완료
14320개 블로그 크롤링 완료
14330개 블로그 크롤링 완료
14340개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://jhculture.tistory.com/1244
14350개 블로그 크롤링 완료
14360개 블로그 크롤링 완료
14370개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://sbook.tistory.com/216
14380개 블로그 크롤링 완료
14390개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://byline.network/2019/11/04-20/
14400개 블로그 크롤링 완료
14410개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://glowglow.tistory.com/273
14420개 블로그 크롤링 완료
14430개 블로그 크롤링 완료
14440개 블로그 크롤링 완료
14450개 블로그 크롤링 완료
14460개 블로그 크롤링 완료
14470개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://oksusyu.tistory.com/30
14480개 블로그 크롤링 완료
14490개 블로그 크롤링 완료
14500개 블로그 크롤링 완료
14510개 블로그 크롤링 완료
14520개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://more-money-no-problems.tistory.com/1239
14530개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://ddusddas.tistory.com/99
14540개 블로그 크롤링 완료
14550개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://sanzinibook.tistory.com/4289
14560개 블로그 크롤링 완료
14570개 블로그 크롤링 완료
네이버 블로그 아닌 url:  http

네이버 블로그 아닌 url:  https://georgienam.tistory.com/33
16140개 블로그 크롤링 완료
16150개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://pju8417.tistory.com/223
네이버 블로그 아닌 url:  https://richpresident.tistory.com/610
네이버 블로그 아닌 url:  https://www.read1825.com/entry/11%EC%9B%94%EC%9D%98-%EC%B6%94%EC%B2%9C%EB%8F%84%EC%84%9C2100-%EC%A3%BD%EC%A7%80-%EC%95%8A%EB%8A%94-%ED%98%BC-%EB%AF%BC%EB%AA%85%EA%B8%B0
16160개 블로그 크롤링 완료
16170개 블로그 크롤링 완료
16180개 블로그 크롤링 완료
16190개 블로그 크롤링 완료
16200개 블로그 크롤링 완료
16210개 블로그 크롤링 완료
16220개 블로그 크롤링 완료
16230개 블로그 크롤링 완료
16240개 블로그 크롤링 완료
16250개 블로그 크롤링 완료
16260개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://alexoidov.tistory.com/1554
16270개 블로그 크롤링 완료
16280개 블로그 크롤링 완료
16290개 블로그 크롤링 완료
네이버 블로그 아닌 url:  http://class1.co.kr/9522193152
16300개 블로그 크롤링 완료
16310개 블로그 크롤링 완료
16320개 블로그 크롤링 완료
16330개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://jeffreysays.tistory.com/6613
16340개 블로그 크롤링 완료
네이버 블로그 아닌 url:  http://thegreenfarm.kr/6550379939
16350개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://univus-k.tistory.com/155
16360개 블

18210개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://panflut0312.tistory.com/3482332
18220개 블로그 크롤링 완료
18230개 블로그 크롤링 완료
18240개 블로그 크롤링 완료
18250개 블로그 크롤링 완료


KeyboardInterrupt: 

In [10]:
len(time_list)

18250

In [11]:
data = {'time' : time_list,
       'review' : review_list,
       'comment' : comment_list,
       'like' : like_list,
       'url' : url_list}

df = pd.DataFrame(data)

print('크롤링 소요 시간: ', time.time()-start)
print('총 {}개 블로그 크롤링 완료'.format(count))
df

크롤링 소요 시간:  78676.07538127899
총 18252개 블로그 크롤링 완료


,time,review,comment,like,url
0,2021. 12. 9. 22:38,눈깔사탕 외계인의 일상 12월9일 (감사)일기:달팽이 책방 ...,[],8,https://blog.naver.com/nunggarsata-alien/22259...
1,2021. 10. 8. 9:47,오늘 뭐 먹지? 손님상차림 생신상 밀키트 간편하게 얼큰순살닭전...,[],4,https://blog.naver.com/ksr_0201/222528331113
2,2018. 7. 24. 20:23,한대비문연 예심/ 본심서평 2018 한양대학교 ...,[],2,https://blog.naver.com/magnetlike/221325722683
3,2021. 8. 24. 23:05,순간남기기 8월 24일의 일상) 항상 고민은 있지! ...,[],10,https://blog.naver.com/sa4333/222482801662
4,2022. 3. 3. 19:52,음슴체 안 쓰기 챌린지 2022.02. 나 너무 많은 일이 잇...,[],8,https://blog.naver.com/nuri926/222663062520
...,...,...,...,...,...
18245,2019. 10. 31. 17:33,신혼일기 신혼일기 22장(슬기편)-오늘의 우리 ...,[],1,https://blog.naver.com/rabbit-bubu/221694245529
18246,2018. 8. 19. 12:18,"2018년 독서일기 김석자, <<현대 일본 문학...",[],[],https://blog.naver.com/iocean74/221341651284
18247,2021. 11. 11. 8:44,"북리뷰 [북리뷰] 다시, 내 인생 하늘혼 ・ ...",[],[],https://blog.naver.com/zenither/222564770021
18248,2018. 2. 3. 16:56,"SNP스킨테일러2기 웹드라마 ""4가지...",[],8,https://blog.naver.com/wodbsqo/221200291543


In [12]:
df.to_csv("1인가구_성백.csv", encoding = "UTF-8-sig")